# Thief Detector
## Build a motion detection algorithm that alarms you when you have an unwanted visitor in your home.

## Steps
- 1. Get the live video feed from your webcam
- 2. Fix a scene (the place you want to monitor) and store it as a reference background image
    - Store the first frame as the reference background frame
- 3. For every frame, check if there is any unwanted object inside the scene you are monitoring
    - Use **Background Subtraction** concept (**cv2.absdiff( )**)
        - Subtract the current frame from the reference background image(frame) to see the changes in the scene
        - If there is enormous amount of pixels distrubed in the subtraction result image
            - unwanted visitor (place is unsafe --> alarm the authorities)
        - If there is no enormous amount of pixels distrubed in the subtraction result image
            - no unwanted visitor (place is safe)
- 4. Output the text **"UNSAFE"** in **red** color on the top right of the frame when there is an intruder in the scene.

In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [18]:
import cv2
import os
# Step 1: Get live video feed from webcam
cap = cv2.VideoCapture(0) # 0 refers to the default camera on your computer

# Step 2: Read first frame, convert to Grayscale and store it as reference background image
ret, frame = cap.read()
ref_background = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 25.0,  (int(cap.get(3)), int(cap.get(4))))

# Start the while loop to read frames from the video capture object
while True:
    # Read a frame from the video capture object
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate the absolute difference between the reference background and the current frame
    diff_frame = cv2.absdiff(ref_background, gray_frame)
    
    # Apply threshold to the difference frame
    ret, thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)
    
    # Find contours in the thresholded frame
    contours, hierarchy = cv2.findContours(thresh_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Check if contour area is large and draw rectangle around the object
    for contour in contours:
        if cv2.contourArea(contour) > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame, "UNSAFE", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
    
    # Write the frame to the video file
    out.write(frame)
    
    # Display the resulting frame
    cv2.imshow('Frame', frame)
    
    # Wait for a key press
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and video writer objects, and destroy all windows
cap.release()
out.release()
cv2.destroyAllWindows()